# Gramener Data Science Entry Level Position Use Case 1

In [2]:
#imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [12]:
datapath = './'
filename = 'nas-pupil-marks.csv'
Data = pd.read_csv(datapath+filename)
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185348 entries, 0 to 185347
Data columns (total 64 columns):
STUID                     185348 non-null int64
State                     185348 non-null object
District                  185348 non-null int64
Gender                    185348 non-null int64
Age                       185348 non-null int64
Category                  185348 non-null int64
Same language             185348 non-null int64
Siblings                  185348 non-null int64
Handicap                  185348 non-null int64
Father edu                185348 non-null int64
Mother edu                185348 non-null int64
Father occupation         185348 non-null int64
Mother occupation         185348 non-null int64
Below poverty             185348 non-null int64
Use calculator            185348 non-null int64
Use computer              166186 non-null object
Use Internet              185348 non-null int64
Use dictionary            185348 non-null int64
Read other books       

Walking into this, we have 3 questions that we hope to answer eventually - 
1. What influences student performance the most?
2. How do girls and boys perform across states?
3. Do students from South India really excel at Math and Science

Before we do deep dives into this, a few issues seem to be cropping up - so let's try to clean the dataset up a little. 
First we should check for nulls - if present, we should attempt to drop them. 

In [20]:
NullsDropped = Data.dropna(axis=0,how='all',inplace=False)
NullsDropped.shape

(185348, 64)

Immediately, there are no rows such that all values are null. The rows which have the most nulls are our metrics of performance, the maths, science, social science and reading scores. This presents a major problem as we can't easily use a statistical measure such as mean of their respective columns to fill the null values, as this might potentially taint our data as we look for correlations. Dropping all the rows with nulls would also not be ideal, as we'd reduce our current number of data points by ~96% as is evidenced below. 

In [21]:
NullsDropped = Data.dropna(axis=0,how='any',inplace=False)
NullsDropped.shape

(8044, 64)

This is problematic, in that dropping rows which have any nulls reduces the number of rows so drastically. 
we're literally left with 4% of our initial dataset. 
One thing that might potentially help is to split our data frame into one frame for each of our metrics, so have a seperate maths column, a separate science column etc. then drop the nulls, this would let us potentially retain close to 40% of our dataset. 
Moreover, we also have 0 levels in our dataset for columns that shouldn't be given 0 levels according to the labels csv file. We also need find a way to deal with those. 

In [23]:
Maths = Data.drop(['Reading %','Science %','Social %'],axis=1)
Maths.dropna(axis=0,how='any',inplace=True)
Maths.shape

(83371, 61)

Similarly, we create a dataset for each metric of performance

In [25]:
Reading = Data.drop(['Maths %','Science %','Social %'],axis=1)
Reading.dropna(axis=0,how='any',inplace=True)

Science = Data.drop(['Maths %','Reading %','Social %'],axis=1)
Science.dropna(axis=0,how='any',inplace=True)

Social = Data.drop(['Maths %','Reading %','Science %'],axis=1)
Social.dropna(axis=0,how='any',inplace=True)

Maths_and_Science = Data.drop(['Reading %','Social %'],axis=1)
Maths_and_Science.dropna(axis=0,how='any',inplace=True)

In [5]:
filename = 'nas-labels.csv'
labels = pd.read_csv(datapath+filename)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 4 columns):
Column    241 non-null object
Name      241 non-null object
Level     241 non-null object
Rename    241 non-null object
dtypes: object(4)
memory usage: 7.6+ KB


In [6]:
zero_allowed = pd.DataFrame(labels.loc[labels['Level'].astype(str) == '0']) 
#Slightly janky, but I didn't want to drop the states and can't convert characters to base 10 ints implicitly.
zero_allowed = zero_allowed['Column'].tolist()
zero_allowed

['Category',
 'Same language',
 'Siblings',
 'Handicap',
 'Father edu',
 'Mother edu',
 'Father occupation',
 'Mother occupation',
 'Below poverty',
 'Subjects']

So, this means that if any row has a zero value for a column that isn't one these displayed above, I need to find a way to either replace it with a mean/median value for that column or just drop that row altogether. 
Personally I prefer dropping if it doesn't reduce my dataset size too much, so let's try that first


In [7]:
temp = pd.DataFrame(columns = Data.columns.values.tolist())
boobs = [item for item in Data.columns.values.tolist() if item not in zero_allowed]
for col in boobs:
#     print(Data.loc[Data[col].astype(str)=='0'])
        temp.append(pd.DataFrame(Data.loc[Data[col].astype(str)=='0']),ignore_index=True)

In [8]:
temp

,STUID,State,District,Gender,Age,Category,Same language,Siblings,Handicap,Father edu,...,Express science views,Watch TV,Read magazine,Read a book,Play games,Help in household,Maths %,Reading %,Science %,Social %
